In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer

from sklearn.svm import SVC, LinearSVC
from sklearn.linear_model import SGDClassifier, RidgeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier

import pandas as pd
from utility import *

In [ ]:
X, y = load_data()
enc = OneHotEncoder(sparse=False)
y = enc.fit_transform(y)
X_imputed = SimpleImputer().fit_transform(X)

In [ ]:
def time_step(data, label, step=10):
    x,y = [],[]
    for i in range(data.shape[0]-(step+1)):
        x.append(data[i:i+step])
        y.append(label[i+step+1])
    return np.array(x),np.array(y)

In [ ]:
def create_dataset(team_name):
    ht = data.loc[data['HomeTeam']==team_name]
    at = data.loc[data['AwayTeam']==team_name]
    team_data = pd.concat([ht,at])
    team_data_label = team_data['FTR']
    team_data_featured = pd.get_dummies(team_data[features])
    return team_data_featured,team_data_label

In [ ]:
team_name = 'Chelsea'
data_f, data_l = create_dataset(team_name)
data_f = SimpleImputer().fit_transform(data_f)
x_ars, y_ars = time_step(data_f, data_l.to_numpy())
print(x_ars.shape)
print(y_ars.shape)

In [ ]:
y_ars = enc.fit_transform(y_ars.reshape(-1,1))

In [ ]:
x_train_lstm, x_test_lstm, y_train_lstm, y_test_lstm = train_test_split(x_ars, y_ars, shuffle=False, test_size=test_size)

In [ ]:
Tx = x_train_lstm.shape[1] #Time steps
Ty = y_train_lstm.shape[1] #Time Steps

# LSTM

In [ ]:
import os

model = None
model_path = './models/lstm/'

# Check se il modello è già salvato
if not os.path.exists(model_path) or os.listdir(model_path):
    os.mkdir("./" + model_path.split("/")[1])
    os.mkdir(model_path)


else:
    model = k.models.load_model(model_path)


if model == None:
    model = k.models.Sequential(
        [
            k.layers.LSTM(64, return_sequences=True),
            k.layers.Dropout(0.4),
            k.layers.Dense(1000, activation="relu"),
            k.layers.Dropout(0.3),
            k.layers.Dense(250, activation="relu"),
            k.layers.Dropout(0.2),
            k.layers.Dense(3, activation="softmax")
        ]
    )

    model.compile(
        loss='categorical_crossentropy',
        optimizer=tf.keras.optimizers.Adam(0.0001),
        metrics=["accuracy"]
    )

    callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=5)

    model.fit(x_train_lstm, y_train_lstm, epochs=10, callbacks=[callback])
    model.save('./models/lstm')

In [ ]:
model.evaluate(x_test_lstm, y_test_lstm)
report(model, x_train_lstm, y_train_lstm, x_test_lstm, y_test_lstm, enc)

## 2 LSTM concatenate + dense e dropout in sequenza

In [ ]:
inputs = k.layers.Input(shape=(10,94))
lstm1 = k.layers.LSTM(100, return_sequences=True, activation="relu")(inputs)
lstm2 = k.layers.LSTM(50, return_sequences=True, activation="relu")(inputs)
concateneted = k.layers.Concatenate()([
    lstm1,
    lstm2
])

out = k.layers.Dropout(0.5)(concateneted)
out = k.layers.Dense(1000, activation="relu")(out)
out = k.layers.Dropout(0.5)(out)
out = k.layers.Dense(250, activation="relu")(out)
out = k.layers.Dropout(0.5)(out)
out = k.layers.Dense(3, activation="softmax")(out)

model = k.models.Model(inputs=inputs, outputs=out)

In [ ]:
model.summary()

In [ ]:
model.compile(
    loss='categorical_crossentropy',
    optimizer=tf.keras.optimizers.Adam(0.0001),
    metrics=["accuracy"]
)

callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=5)

model.fit(x_train_lstm, y_train_lstm, epochs=500, callbacks=[callback])

In [ ]:
model.evaluate(x_test_lstm, y_test_lstm)
report(model, x_train_lstm, y_train_lstm, x_test_lstm, y_test_lstm, enc)

### Fit
loss: 0.9373 - accuracy: 0.5609
### Evaluate
loss: 0.9192 - accuracy: 0.5930
### Classification report

| | precision | recall | f1-score | support |
| - | ----------- | ------ | -------- | ------- |
| A | 0.609 | 0.614 | 0.612 | 345 |
| D | 0.321 | 0.201 | 0.247 | 254 |
| H | 0.652 | 0.763 | 0.704 | 541 |
| accuracy | |  | 0.593 | 1140 |
| macro avg | 0.527 | 0.526 | 0.521 | 1140 |
| weighted avg | 0.565 | 0.593 | 0.574 | 1140 |

## LSTM in parallelo

In [ ]:
inputs = k.layers.Input(shape=(10,94))
x = []
for t in range(Tx):
    module = k.layers.LSTM((t+1)*10, return_sequences=True)(inputs)
    module = k.layers.Dropout(0.7)(module)
    x.append(module)
x = k.layers.Concatenate()(x)

out = k.layers.Dense(1000, activation="relu")(x)
out = k.layers.Dropout(0.7)(out)
out = k.layers.Dense(250, activation="relu")(out)
out = k.layers.Dense(3, activation="softmax")(out)

txLstm = k.models.Model(inputs=inputs, outputs=out)

In [ ]:
txLstm.summary()

In [ ]:
txLstm.compile(
    loss='categorical_crossentropy',
    optimizer=tf.keras.optimizers.Adam(0.0001),
    metrics=["accuracy"]
)

callback = tf.keras.callbacks.EarlyStopping(monitor='accuracy', patience=3)

txLstm.fit(x_train_lstm, y_train_lstm, epochs=500, callbacks=[callback], batch_size=16)

In [ ]:
txLstm.evaluate(y_test_lstm, y_test_lstm)
report(txLstm, x_train_lstm, y_train_lstm, x_test_lstm, y_test_lstm, enc)

# GRU

In [ ]:
gru = k.models.Sequential([
    k.layers.GRU(100, return_sequences=True, activation="relu"),
    k.layers.Dropout(0.4),
    k.layers.Dense(1000, activation="relu"),
    k.layers.Dropout(0.3),
    k.layers.Dense(250, activation="relu"),
    k.layers.Dropout(0.2),
    k.layers.Dense(3, activation="softmax")
])

In [ ]:
gru(x_train_lstm)
gru.summary()

In [ ]:
gru.compile(
    loss='categorical_crossentropy',
    optimizer=tf.keras.optimizers.Adam(0.0001),
    metrics=["accuracy"]
)

callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=5)

gru.fit(x_train_lstm, y_train_lstm, epochs=500, callbacks=[callback])

In [ ]:
gru.evaluate(x_test_lstm, y_test_lstm)
report(gru, x_train_lstm, y_train_lstm, x_test_lstm, y_test_lstm, enc)

# Time distributed

In [ ]:
model = k.models.Sequential(
    [
        k.layers.GRU(128, dropout=0.5, return_sequences=True),
        k.layers.Flatten(),
        k.layers.Dense(40, activation="relu"),
        k.layers.Dropout(0.4),
        k.layers.Dense(20, activation="relu"),
        k.layers.Dropout(0.2),
        k.layers.Dense(10, activation="relu"),
        k.layers.Dense(3, activation="softmax")
    ]
)

In [ ]:
model.compile(
    loss='categorical_crossentropy',
    optimizer=tf.keras.optimizers.Adam(0.0001),
    metrics=["accuracy"]
)
model(x_train_lstm)
model.summary()

In [ ]:
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=10)

model.fit(x_train_lstm, y_train_lstm, epochs=1000, callbacks=[callback])

In [ ]:
model.evaluate(x_test_lstm, y_test_lstm)

report(model, x_train_lstm, y_train_lstm, "train", enc)
report(model, x_test_lstm, y_test_lstm, "test", enc)